# Analysis of the results for Experiment 3

In [1]:
import pandas as pd
import numpy as np

In [29]:
dtypes = {'Dataset': 'category',
          ' Model': 'category',
          ' Seed': np.int32,
          ' Aug': 'category',
          ' BestEpoch': np.int32,
          ' TestAcc': np.float64,
          ' TestLoss': np.float64,
          ' ValAcc': np.float64,
          ' ValLoss': np.float64,
          ' Device': 'category',
          ' Time': np.float64
          }

In [30]:
results = pd.read_csv('../results/train_log_exp3.csv', header=0, dtype=dtypes)
columns = [x.strip() for x in list(dtypes.keys())]
results.columns = columns

In [31]:
results.head()

,Dataset,Model,Seed,Aug,BestEpoch,TestAcc,TestLoss,ValAcc,ValLoss,Device,Time
0,IMDB-BINARY,GCN,1314,Vanilla,136,0.700,0.7624,0.68,1.1394,cpu,275.95
1,IMDB-BINARY,GCN,1314,G-Mixup,6,0.760,0.5180,0.68,0.5254,cpu,288.35
2,IMDB-BINARY,GCN,1314,Subgraph,36,0.745,1.3456,0.71,1.5321,cpu,74.98
3,IMDB-BINARY,GCN,1314,DropEdge,3,0.715,0.6268,0.69,0.6022,cpu,123.67
4,IMDB-BINARY,GCN,1314,DropNode,7,0.770,0.4938,0.72,0.5869,cpu,81.34


In [45]:
grouped = results.groupby(['Dataset', 'Model', 'Aug']).agg([np.mean, np.std, np.count_nonzero]).loc[:,['TestAcc']]
grouped

TestAcc                        
                                       mean       std count_nonzero
Dataset     Model       Aug                                        
IMDB-BINARY  DiffPool    DropEdge  0.719000  0.025473          10.0
                         DropNode  0.710500  0.022540          10.0
                         G-Mixup   0.709500  0.031486          10.0
                         Subgraph  0.701500  0.035438          10.0
                         Vanilla   0.721500  0.020419          10.0
             GCN         DropEdge  0.717000  0.024290          10.0
                         DropNode  0.720500  0.033620          10.0
                         G-Mixup   0.719000  0.029231          10.0
                         Subgraph  0.720000  0.039511          10.0
                         Vanilla   0.714000  0.022211          10.0
             GIN         DropEdge  0.712000  0.031640          10.0
                         DropNode  0.703000  0.036301          10.0
                         G-Mixup   0.704500  0.039472          10.0
                         Subgraph  0.698000  0.033267          10.0
                         Vanilla   0.720000  0.029814          10.0
             MinCutPool  DropEdge  0.715000  0.036968          10.0
                         DropNode  0.715000  0.029250          10.0
                         G-Mixup   0.712500  0.020582          10.0
                         Subgraph  0.710500  0.030226          10.0
                         Vanilla   0.715500  0.026714          10.0
             TopKPool    DropEdge  0.687000  0.041780          10.0
                         DropNode  0.716000  0.017607          10.0
                         G-Mixup   0.708000  0.027406          10.0
                         Subgraph  0.694000  0.047364          10.0
                         Vanilla   0.706500  0.031977          10.0
IMDB-MULTI   DiffPool    DropEdge       NaN       NaN           NaN
                         DropNode       NaN       NaN           NaN
                         G-Mixup        NaN       NaN           NaN
                         Subgraph       NaN       NaN           NaN
                         Vanilla        NaN       NaN           NaN
             GCN         DropEdge  0.513333  0.020276           3.0
                         DropNode  0.508889  0.025240           3.0
                         G-Mixup   0.524445  0.006939           3.0
                         Subgraph  0.486667  0.031798           3.0
                         Vanilla   0.520000  0.018559           3.0
             GIN         DropEdge       NaN       NaN           NaN
                         DropNode       NaN       NaN           NaN
                         G-Mixup        NaN       NaN           NaN
                         Subgraph       NaN       NaN           NaN
                         Vanilla        NaN       NaN           NaN
             MinCutPool  DropEdge       NaN       NaN           NaN
                         DropNode       NaN       NaN           NaN
                         G-Mixup        NaN       NaN           NaN
                         Subgraph       NaN       NaN           NaN
                         Vanilla        NaN       NaN           NaN
             TopKPool    DropEdge       NaN       NaN           NaN
                         DropNode       NaN       NaN           NaN
                         G-Mixup        NaN       NaN           NaN
                         Subgraph       NaN       NaN           NaN
                         Vanilla        NaN       NaN           NaN

### Analysis of train/val/test losses